In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("taxi-fare-prediction").getOrCreate()
spark

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/01 09:05:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
directory = "/home/ubuntu/working/spark/data"
trip_files = "/trips/*"

In [3]:
trips_df = spark.read.csv(f"file:///{directory}/{trip_files}", inferSchema=True, header=True)
trips_df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [4]:
trips_df.createOrReplaceTempView("trips")

# 데이터 정제

In [5]:
# 데이터 정제
query = """
SELECT
    t.trip_distance,
    t.total_amount

FROM trips t

WHERE t.total_amount < 200
  AND t.total_amount > 0
  AND t.passenger_count < 5
  AND TO_DATE(t.tpep_pickup_datetime) >= '2021-01-01'
  AND TO_DATE(t.tpep_pickup_datetime) < '2021-08-01'
  AND t.trip_distance < 10
  AND t.trip_distance > 0
"""

data_df = spark.sql(query)

In [6]:
data_df.show(5)

+-------------+------------+
|trip_distance|total_amount|
+-------------+------------+
|         1.13|       11.16|
|         2.68|       18.59|
|          9.7|        32.3|
|          9.3|       43.67|
|         9.58|        46.1|
+-------------+------------+
only showing top 5 rows



# Train/Test 분할

In [7]:
train_sdf, test_sdf = data_df.randomSplit([0.8, 0.2], seed=42)

In [8]:
train_sdf.cache()

DataFrame[trip_distance: double, total_amount: double]

# 특성 벡터 어셈블링
- `feature` 가 하나만 있어도 벡터 형식으로 데이터를 뭉쳐야 한다

In [11]:
from pyspark.ml.feature import VectorAssembler

vec_assembler = VectorAssembler(
    inputCols = ["trip_distance"],
    outputCol = 'features'
)

In [12]:
vec_train_sdf = vec_assembler.transform(train_sdf)
vec_train_sdf.show(5)

+-------------+------------+--------+
|trip_distance|total_amount|features|
+-------------+------------+--------+
|         0.01|        3.05|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
+-------------+------------+--------+
only showing top 5 rows



# 모델 생성 및 훈련

In [13]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(
    maxIter=50,
    featuresCol="features",
    labelCol="total_amount"
)

In [14]:
lr_model = lr.fit(vec_train_sdf)

24/04/01 09:38:59 WARN Instrumentation: [bf551c71] regParam is zero, which might cause numerical instability and overfitting.
24/04/01 09:39:00 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/04/01 09:39:00 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
24/04/01 09:39:52 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


# 예측

In [16]:
vec_test_sdf = vec_assembler.transform(test_sdf)
vec_test_sdf.show(5)

+-------------+------------+--------+
|trip_distance|total_amount|features|
+-------------+------------+--------+
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
+-------------+------------+--------+
only showing top 5 rows



In [17]:
predictions = lr_model.transform(vec_test_sdf)
predictions.show(5)

+-------------+------------+--------+-----------------+
|trip_distance|total_amount|features|       prediction|
+-------------+------------+--------+-----------------+
|         0.01|         3.3|  [0.01]|7.966597441113208|
|         0.01|         3.3|  [0.01]|7.966597441113208|
|         0.01|         3.3|  [0.01]|7.966597441113208|
|         0.01|         3.3|  [0.01]|7.966597441113208|
|         0.01|         3.3|  [0.01]|7.966597441113208|
+-------------+------------+--------+-----------------+
only showing top 5 rows



# 평가

In [18]:
lr_model.summary.rootMeanSquaredError

3.510945426773181

In [19]:
lr_model.summary.r2

0.763032284678061

In [20]:
spark.stop()